## 引言
通过前面两篇文章[欺诈文本分类微调（二）：生成正向数据集](https://golfxiao.blog.csdn.net/article/details/141274990)和[欺诈文本分类微调（三）：生成反向数据集](https://golfxiao.blog.csdn.net/article/details/141301705)我们已经生成了正向和反向的基础数据集，不过这些数据集还是比较原始的长篇对话，并不适合直接用于特定任务的指令微调训练，大概还需要进行以下几方面处理：
- 区分长度作切割：切割出不同长度的对话文本
- 区分标签重分布：正、反向数据集的均衡分布
- 区分用途作切分：训练、验证、测试集的构造

## 按长度切割

之所以要对一个完整的案例进行切割，是为了尽量贴合实际场景。一方面是一场会议比较长，我们不大可能把整个会议的ASR文本一次给到大模型做欺诈检测； 另一方面是为了做到实时检测，也决定了我们拿到的必定是一段对话文本，而不是整个会议ASR。

#### 加载数据文件

复用前面一篇文章的工具函数。

In [10]:
import pandas as pd
import os
import json

def filename(path):
    filename_with_ext = os.path.basename(path)
    filename, extention = os.path.splitext(filename_with_ext)
    return filename

def get_files(directory, extension='.csv'):
    files = [f for f in os.listdir(directory) if f.endswith(extension)]
    return files

def print_json(obj):
    print(json.dumps(obj, indent=4, ensure_ascii=False))

input_dir = "../dataset/csv_dialogs"
files = get_files(input_dir)
files

['虚假购物诈骗(200).csv',
 '网络赌博诈骗(200).csv',
 '“杀猪盘” 诈骗(200).csv',
 '虚拟货币诈骗(200).csv',
 '冒充客服诈骗(200).csv',
 '传销诈骗(140).csv',
 '短信诈骗(200).csv',
 '网络贷款诈骗(162).csv',
 '微商代理诈骗(200).csv',
 '二手交易诈骗(200).csv',
 '兼职刷单诈骗(140).csv',
 'meeting_label_false.csv',
 '投资诈骗(200).csv']

In [11]:
df = pd.read_csv(os.path.join(input_dir, files[10]))
df.head(20)

,case,speaker,content,is_fraud
0,案例-0,刘艳,您好，我是某某网的兼职客服，我们这里有一个刷单任务，可以轻松赚钱，您有兴趣吗？,True
1,案例-0,吕女士,您好，刷单任务是什么？怎么赚钱的？,False
2,案例-0,刘艳,刷单就是在指定的平台上完成一些简单的操作，比如点赞、评论等。完成后我们会按照任务金额的5%-...,True
3,案例-0,吕女士,真的这么简单就能赚钱？,False
4,案例-0,刘艳,当然，我们已经有很多人通过这个兼职赚到了钱，这里还有大家的截图和返利记录。您只需要下载我们提...,True
5,案例-0,吕女士,那我可以试试看吗？,False
6,案例-0,刘艳,当然可以，这是任务APP的下载链接，请先注册一个账号。,True
7,案例-0,吕女士,我已经注册好了，接下来怎么做？,False
8,案例-0,刘艳,现在您可以先试做几个小任务，比如点赞和评论，这些任务金额较小，但我们会马上返利给您，您看一下...,True
9,案例-0,吕女士,我已经完成了任务，看到了返利，真的挺简单的。,False


#### 对话集切割

定义数据集中的列名，以及任务中的提示词指令。

In [12]:
group_column = 'case'
speaker_column = 'speaker'
content_column = 'content'
is_fraud_column = 'is_fraud'
instruction = "下面是一段对话文本, 请分析对话内容是否有诈骗风险，只以json格式输出你的判断结果(is_fraud: true/false)。"

##### 工具函数定义

定义几个工具函数用于辅助对话集分割，
- find_fraud_speaker：用于找出案例中属于诈骗角色的发言人，对于正向（欺诈）数据集，里面必定有一个发言人属于诈骗者。
- total_chars: 用于统计一组对话集的长度，用于实时判断分割后对话集的长度是否超限。
- to_train_data: 用于将一组对话集转换为一条训练数据。

In [13]:
def find_fraud_speaker(df: pd.DataFrame):
    # 按照speaker列进行分组，并统计每个分组下is_fraud=True的数量
    fraud_counts = df.groupby(speaker_column)[is_fraud_column].sum()
    # 特殊情况处理：对于没有is_fraud=True的对话集，则没有诈骗者
    if fraud_counts.max() == 0:
        return ''
    # 统计欺诈最多的speaker即认为是诈骗者
    return fraud_counts.idxmax()

def total_chars(strs: list):
    return sum(len(s) for s in strs)

def to_train_data(dialog:list, label, instruction, fraud_speaker=''):
    content = "\n".join(dialog)
    return {"input": content, "label": label, "fraud_speaker": fraud_speaker, "instruction": instruction}


定义一个函数split_dialog用于处理单个案例的分割，按照指定的长度将一个案例分割成多段对话，每段对话作为一条训练数据。

In [14]:

def split_dialog(dialog: pd.DataFrame, max_length):
    small_dialogs = []
    current_segment = []
    current_label = False
    fraud_speaker = find_fraud_speaker(dialog)
    # 遍历dataframe中的所有数据行
    for _, item in dialog.iterrows():
        statement = f"{item[speaker_column]}: {item[content_column]}"
        if max_length <= 0:
            current_segment.append(statement)
        elif total_chars(current_segment) + len(statement) < max_length:
            current_segment.append(statement)
        else:
            small_dialogs.append(to_train_data(current_segment, current_label, instruction, fraud_speaker))
            current_label = False
            current_segment[:] = [statement]
        current_label = True if item[is_fraud_column] == True else current_label
    # 处理最后一段剩余的对话，为避免内容太碎，限制对话集在两条及以上时才有效
    if len(current_segment) >= 2:
        small_dialogs.append(to_train_data(current_segment, current_label, instruction, fraud_speaker))
    return small_dialogs

##### 分组操作

由于不同的案例不能分割到不同的对话集中，因此，在分割对话集之前，要先按案例进行分组。

预览下groupby分组后的效果。

In [15]:
import json

df = pd.read_csv(os.path.join(input_dir, files[10]))
grouped = df.groupby(group_column)
grouped.size()

case
案例-0      22
案例-1      22
案例-10     14
案例-100    17
案例-101    18
          ..
案例-95     22
案例-96     28
案例-97     15
案例-98     10
案例-99     21
Length: 139, dtype: int64

有两种方式对groupby分组后的DataFrame进行迭代：
1. 使用grouped.groups对键进行迭代，
2. 直接使用for name, group in grouped:来迭代分组名和对应的DataFrame片段

In [16]:
for name, group in grouped:
    print(name, group)
    print("threshold = 100 chars")
    print_json(split_dialog(group, 100))

    print("threshold = 300 chars")
    print_json(split_dialog(group, 300))

    print("threshold = 500 chars")
    print_json(split_dialog(group, 500))

    break

案例-0     case speaker                                            content  is_fraud
0   案例-0      刘艳            您好，我是某某网的兼职客服，我们这里有一个刷单任务，可以轻松赚钱，您有兴趣吗？      True
1   案例-0     吕女士                                  您好，刷单任务是什么？怎么赚钱的？     False
2   案例-0      刘艳  刷单就是在指定的平台上完成一些简单的操作，比如点赞、评论等。完成后我们会按照任务金额的5%-...      True
3   案例-0     吕女士                                        真的这么简单就能赚钱？     False
4   案例-0      刘艳  当然，我们已经有很多人通过这个兼职赚到了钱，这里还有大家的截图和返利记录。您只需要下载我们提...      True
5   案例-0     吕女士                                          那我可以试试看吗？     False
6   案例-0      刘艳                        当然可以，这是任务APP的下载链接，请先注册一个账号。      True
7   案例-0     吕女士                                    我已经注册好了，接下来怎么做？     False
8   案例-0      刘艳  现在您可以先试做几个小任务，比如点赞和评论，这些任务金额较小，但我们会马上返利给您，您看一下...      True
9   案例-0     吕女士                             我已经完成了任务，看到了返利，真的挺简单的。     False
10  案例-0      刘艳    看到吧，非常简单！现在我们有一些大单任务，返利更高，只需要投入更多资金刷单，您是不是很赚到了？      True
11  案例-0     吕女士                                           

#### 主循环

convert_to_jsonl：单个数据文件处理，将csv格式的对话集文件，转换成jsonl格式的对话集文件，转换过程：
1. 读取输入文件，并按照第一列case（案例名称）进行分组，case相同的对话属于同一个案例。
2. 循环对每个案例应用split_dialog函数，将长对话集分割成短对话集并转换为json格式。
3. 将所有分割后的短对话集以jsonl的格式保存。

> 注：我们在分割对话集时，必须在一个案例内部进行分割，确保一条训练数据中不会出现不同案例的交叉。

In [17]:
def convert_to_jsonl(input_path, output_path, max_length=200):
    df = pd.read_csv(input_path)
    # 按照某列的数据进行分组
    grouped = df.groupby(group_column)
    # 分割对话集，长对话按照指定长度分割成短对话
    all_dialogs = []
    for _, group in grouped:
        small_dialogs = split_dialog(group, max_length)
        all_dialogs.extend(small_dialogs)
    train_dataset = pd.DataFrame(all_dialogs)
    # orient="records" 表示dataset中的每一行是一个json对象
    # lines=True 表示每个json对象写入文件时占一行
    train_dataset.to_json(output_path, orient="records", lines=True, force_ascii=False)

all_to_jsonl用于将一个目录下的所有csv对话集文件都转换为jsonl格式的文件。

In [18]:
output_dir = "../dataset/fraud/jsonl"
os.makedirs(output_dir, exist_ok=True)

def all_to_jsonl(input_dir, output_dir, max_length):
    max_length = 0 if max_length < 0 else max_length
    files = get_files(input_dir, extension=".csv")
    for file in files:
        input_path = os.path.join(input_dir, file)
        output_file = f"{filename(input_path)}_train_{max_length}.jsonl"
        output_path = os.path.join(output_dir, output_file)
        convert_to_jsonl(input_path, output_path, max_length)
        print(f"convert {file} to {output_file}")

按照100字符的长度进行切割，将csv转换为jsonl。

In [19]:
all_to_jsonl(input_dir, output_dir, 100)

convert 虚假购物诈骗(200).csv to 虚假购物诈骗(200)_train_100.jsonl
convert 网络赌博诈骗(200).csv to 网络赌博诈骗(200)_train_100.jsonl
convert “杀猪盘” 诈骗(200).csv to “杀猪盘” 诈骗(200)_train_100.jsonl
convert 虚拟货币诈骗(200).csv to 虚拟货币诈骗(200)_train_100.jsonl
convert 冒充客服诈骗(200).csv to 冒充客服诈骗(200)_train_100.jsonl
convert 传销诈骗(140).csv to 传销诈骗(140)_train_100.jsonl
convert 短信诈骗(200).csv to 短信诈骗(200)_train_100.jsonl
convert 网络贷款诈骗(162).csv to 网络贷款诈骗(162)_train_100.jsonl
convert 微商代理诈骗(200).csv to 微商代理诈骗(200)_train_100.jsonl
convert 二手交易诈骗(200).csv to 二手交易诈骗(200)_train_100.jsonl
convert 兼职刷单诈骗(140).csv to 兼职刷单诈骗(140)_train_100.jsonl
convert meeting_label_false.csv to meeting_label_false_train_100.jsonl
convert 投资诈骗(200).csv to 投资诈骗(200)_train_100.jsonl


按照300字符的长度进行切割，将csv转换为jsonl。

In [20]:

all_to_jsonl(input_dir, output_dir, 300)

convert 虚假购物诈骗(200).csv to 虚假购物诈骗(200)_train_300.jsonl
convert 网络赌博诈骗(200).csv to 网络赌博诈骗(200)_train_300.jsonl
convert “杀猪盘” 诈骗(200).csv to “杀猪盘” 诈骗(200)_train_300.jsonl
convert 虚拟货币诈骗(200).csv to 虚拟货币诈骗(200)_train_300.jsonl
convert 冒充客服诈骗(200).csv to 冒充客服诈骗(200)_train_300.jsonl
convert 传销诈骗(140).csv to 传销诈骗(140)_train_300.jsonl
convert 短信诈骗(200).csv to 短信诈骗(200)_train_300.jsonl
convert 网络贷款诈骗(162).csv to 网络贷款诈骗(162)_train_300.jsonl
convert 微商代理诈骗(200).csv to 微商代理诈骗(200)_train_300.jsonl
convert 二手交易诈骗(200).csv to 二手交易诈骗(200)_train_300.jsonl
convert 兼职刷单诈骗(140).csv to 兼职刷单诈骗(140)_train_300.jsonl
convert meeting_label_false.csv to meeting_label_false_train_300.jsonl
convert 投资诈骗(200).csv to 投资诈骗(200)_train_300.jsonl


按照500字符的长度进行切割，将csv转换为jsonl。

In [21]:
all_to_jsonl(input_dir, output_dir, 500)

convert 虚假购物诈骗(200).csv to 虚假购物诈骗(200)_train_500.jsonl
convert 网络赌博诈骗(200).csv to 网络赌博诈骗(200)_train_500.jsonl
convert “杀猪盘” 诈骗(200).csv to “杀猪盘” 诈骗(200)_train_500.jsonl
convert 虚拟货币诈骗(200).csv to 虚拟货币诈骗(200)_train_500.jsonl
convert 冒充客服诈骗(200).csv to 冒充客服诈骗(200)_train_500.jsonl
convert 传销诈骗(140).csv to 传销诈骗(140)_train_500.jsonl
convert 短信诈骗(200).csv to 短信诈骗(200)_train_500.jsonl
convert 网络贷款诈骗(162).csv to 网络贷款诈骗(162)_train_500.jsonl
convert 微商代理诈骗(200).csv to 微商代理诈骗(200)_train_500.jsonl
convert 二手交易诈骗(200).csv to 二手交易诈骗(200)_train_500.jsonl
convert 兼职刷单诈骗(140).csv to 兼职刷单诈骗(140)_train_500.jsonl
convert meeting_label_false.csv to meeting_label_false_train_500.jsonl
convert 投资诈骗(200).csv to 投资诈骗(200)_train_500.jsonl


这样，所有的案例都被切割成了100、300、500长度的小对话集。

In [22]:
!ls -l ../dataset/fraud/jsonl

total 22144
-rw-r--r-- 1 root root 2899397 Mar 15 22:55  meeting_label_false_train_100.jsonl
-rw-r--r-- 1 root root 2253870 Mar 15 22:55  meeting_label_false_train_300.jsonl
-rw-r--r-- 1 root root 2031157 Mar 15 22:55  meeting_label_false_train_500.jsonl
-rw-r--r-- 1 root root  715360 Mar 15 22:55 '“杀猪盘” 诈骗(200)_train_100.jsonl'
-rw-r--r-- 1 root root  491999 Mar 15 22:55 '“杀猪盘” 诈骗(200)_train_300.jsonl'
-rw-r--r-- 1 root root  455843 Mar 15 22:55 '“杀猪盘” 诈骗(200)_train_500.jsonl'
-rw-r--r-- 1 root root  504839 Mar 15 22:55 '二手交易诈骗(200)_train_100.jsonl'
-rw-r--r-- 1 root root  351986 Mar 15 22:55 '二手交易诈骗(200)_train_300.jsonl'
-rw-r--r-- 1 root root  322703 Mar 15 22:55 '二手交易诈骗(200)_train_500.jsonl'
-rw-r--r-- 1 root root  314864 Mar 15 22:55 '传销诈骗(140)_train_100.jsonl'
-rw-r--r-- 1 root root  222343 Mar 15 22:55 '传销诈骗(140)_train_300.jsonl'
-rw-r--r-- 1 root root  205191 Mar 15 22:55 '传销诈骗(140)_train_500.jsonl'
-rw-r--r-- 1 root root  463026 Mar 15 22:55 '兼职刷单诈骗(140)_train_100.jsonl'
-rw-r

## 构造训练/验证/测试集


#### 加载数据集
加载指定目录下的所有jsonl文件，获得整个dataset，此dataset尚未区分训练集和测试集。

In [23]:
def load_jsonl_files(directory):
    dataset = []
    for filename in os.listdir(directory):
        if filename.endswith('.jsonl'):
            file_path = os.path.join(directory, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                data = [json.loads(line) for line in file]
                dataset.extend(data)
    return dataset

# 设置你的目录路径
input_path = "../dataset/fraud/jsonl"
dataset = load_jsonl_files(input_path)
dataset[0]

{'input': '张强: 你好，我是张强，最近在关注金融投资，看到了您的帖子，感觉很有共鸣，不知道您是否有兴趣一起交流和投资呢？\n李明: 你好张强，我也对投资很感兴趣。您通常投资哪些领域呢？',
 'label': True,
 'fraud_speaker': '张强',
 'instruction': '下面是一段对话文本, 请分析对话内容是否有诈骗风险，只以json格式输出你的判断结果(is_fraud: true/false)。'}

规范化数据集

In [24]:
def normalize_record(record):
    return {
        #"instruction": "你是一个分析诈骗案例的专家，你的任务是分析下面对话内容是否存在经济诈骗(is_fraud:<bool>)，如果存在经济诈骗，请找出正在进行诈骗行为的发言者姓名(fraud_speaker:<str>)，并给出你的分析理由(reason:<str>)，最后以json格式输出。",
        "instruction": str(record.get("instruction", "")),
        "input": str(record.get("input", "")),
        "label": bool(record.get("label", False)),
        # "fraud_speaker": str(record.get("fraud_speaker", "")),
        # "reason": str(record.get("reason", ""))
    }

dataset = [normalize_record(item) for item in dataset]
dataset[10000]

{'instruction': '下面是一段对话文本, 请分析对话内容是否有诈骗风险，只以json格式输出你的判断结果(is_fraud: true/false)。',
 'input': '雷强: 请您将开户短信验证码发给我，我们会为您激活账户。\n张林: 我已经发给您了。\n雷强: 不错，预计很快就会收到贷款。\n张林: 为什么我的银行卡里转出了17673.17元，这是什么情况？',
 'label': True}

In [27]:
# speaker_none_count = sum(d['fraud_speaker'] == None for d in dataset)
# reason_none_count = sum(d['reason'] == None for d in dataset)
# speaker_none_count, reason_none_count

#### 均衡数据集

分别统计数据集中正向标签和反向标签的数量：

In [28]:
true_count = sum(d['label'] == True for d in dataset)
false_count = sum(d['label'] == False for d in dataset)

print(f"true_data: {true_count}, false_data: {false_count}")

true_data: 19538, false_data: 13868


可以看到，目前数据集中的正、反标签的数据比例接近`3:2`，并不均衡。

在二分类任务中，正向标签和反向标签之间的数据集均衡是比较重要的，我们有必要采取一些措施来平衡数据集。大概有以下两类方法：
1. 过采样少数类：通过生成新的少数类样本使其与多数类样本数量相当。
2. 欠采样多数类：减少多数类样本的数量以平衡数据集。

为了快速验证，这里暂时采用欠采样多数类，也就是减少正向标签的数量来平衡数据集。

> 注：不均衡数据集会带来以下影响：
> 1. 模型偏向于预测多数类：如果数据集中某一类别的样本明显多于另一类别，模型在训练时可能会过于偏向于识别多数类，而忽视少数类的特征（即使少数类也很重要）。
> 2. 不准确的指标评估：在不均衡的数据集中，模型的准确率可能很高，但这并不表明模型表现良好，因为它可能只是在简单地预测为多数类，这样的模型往往在少数类的召回率指标上表现差。

In [29]:
from random import sample

def rebalance_dataset(dataset, debug=False):
    # 按标签值分割数据集
    true_data = [d for d in dataset if d['label'] == True]
    false_data = [d for d in dataset if d['label'] == False]
    # 欠采样多数类，这里有两种可能：
    # 1. 如果true_data > false_data，则以false_data的数量为基准来欠采样true_data，使其与false_data的数量相等.
    # 2. 反过来，则以true_data的数量为基准来欠采样false_data，使其与true_data的数量相等。.
    if len(true_data) > len(false_data):
        true_data = sample(true_data, len(false_data))
    else:
        false_data = sample(false_data, len(true_data))
    print(f"balanced_true_data: {len(true_data)}, balanced_false_data: {len(false_data)}") if debug==True else None
    return true_data + false_data

In [30]:
balanced_dataset = rebalance_dataset(dataset)
true_count = sum(d['label'] == True for d in balanced_dataset)
false_count = sum(d['label'] == False for d in balanced_dataset)

print(f"true_data after balance: {true_count}, false_data after balance: {false_count}")

true_data after balance: 13868, false_data after balance: 13868


#### 切分数据集
定义一个split_dataset函数用于按照指定的比例来分割训练集和测试集。基本实现方法：
1. 先使用random.shuffle()将数据打乱顺序，以确保不同长度、不同标签的数据分布更均匀。
2. 按照指定比例先切割训练集，剩下的就是测试集。

> 注：虽然这里将split_dataset定义为切分训练集和测试集，但理论上它可用于任意目的的数据集切分，例如后面会提到的验证集和测试集切分。

In [31]:
import os
import json
import random

def split_dataset(lines, train_ratio=0.8):
    random.shuffle(lines)
    split_index = int(len(lines) * train_ratio)
    train_data = lines[:split_index]
    test_data = lines[split_index:]
    return train_data, test_data

最后，定义一个总的方法make_train_eval_test，来完成训练集、验证集、测试集的构造。切分比例如下：
1. 训练集默认为0.8，支持传参调整。
2. 验证集和测试集比例先固定1:1，平分除训练集以外剩下的部分。

In [32]:
def make_train_eval_test(dataset, train_ratio=0.8):
    train_data, temp_data = split_dataset(dataset, train_ratio)
    eval_data, test_data = split_dataset(temp_data, 0.5)
    return train_data, eval_data, test_data

train_data, eval_data, test_data = make_train_eval_test(balanced_dataset, train_ratio=0.8)
len(train_data), len(eval_data), len(test_data)

(22188, 2774, 2774)

#### 保存数据集

分别保存训练集和测试集数据。

In [33]:
def save_data(dataset, file_path):
    df = pd.DataFrame(dataset)
    df.to_json(file_path, orient="records", lines=True, force_ascii=False)

In [34]:

output_path = "../dataset/fraud/train_test"
os.makedirs(output_path, exist_ok=True)

save_data(train_data, os.path.join(output_path, 'train0902.jsonl'))
save_data(eval_data, os.path.join(output_path, 'eval0902.jsonl'))
save_data(test_data, os.path.join(output_path, 'test0902.jsonl'))

使用pandas的read_json函数预览下生成的数据：
> 注：指定lines=True可以支持jsonl格式。

In [35]:
pd.read_json(os.path.join(output_path, 'train0902.jsonl'), lines=True).head()

,instruction,input,label
0,"下面是一段对话文本, 请分析对话内容是否有诈骗风险，只以json格式输出你的判断结果(is_...",覃峥嵘: 是这样的，我手头有些固定资金要用，但是暂时没办法动用，所以才来找你的。\n亲朋好友...,True
1,"下面是一段对话文本, 请分析对话内容是否有诈骗风险，只以json格式输出你的判断结果(is_...",发言人6: 就是因为别人好像除了我们这块应用以外，它还有一些其他领域的应用。这块咱们有了解吗...,False
2,"下面是一段对话文本, 请分析对话内容是否有诈骗风险，只以json格式输出你的判断结果(is_...",王伟: 你好，我是王伟，在这个婚恋网站上看到你的资料，觉得你很有趣，想认识你一下。\n金玲:...,False
3,"下面是一段对话文本, 请分析对话内容是否有诈骗风险，只以json格式输出你的判断结果(is_...",张某: 哦，这可能是系统的一个小问题，你放心，手机马上发货。要是有什么问题，你可以联系我们的...,True
4,"下面是一段对话文本, 请分析对话内容是否有诈骗风险，只以json格式输出你的判断结果(is_...",张佳: 王军先生，有时候我们的系统会出现一些错误，可能会给您带来误解。为了确认您的身份，您需...,True


**小结**：本文按100、300、500分别构造了不同长度的对话上下文，又通过欠采样来平衡正、负标签数据，最终对平衡后的数据作切分，得到了训练集和测试集。

## 相关阅读
- [欺诈文本分类微调（二）：生成正向数据集](https://golfxiao.blog.csdn.net/article/details/141274990)
- [欺诈文本分类微调（三）：生成反向数据集](https://golfxiao.blog.csdn.net/article/details/141301705)